### Extract timing files

In [1]:
#r "nuget: System.Data.SQLite, *-*"
#r "nuget: Microsoft.DotNet.Interactive.ExtensionLab, *-*"
#r "nuget: Dapper, *-*"

using System.IO;
using System.Text.Json;
using System.Text.Encodings.Web;
using System.Text.Unicode;
using System.Data.SQLite;
using Dapper;

var jsonSerializerOptions = new JsonSerializerOptions
{
    Encoder = JavaScriptEncoder.Create(UnicodeRanges.All),
    //WriteIndented = true
};

#### Recitation Timing (Word by word)

In [3]:
var dbPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\";

record WbwRecitationTiming(long sura, long ayah, long time, string words);

bool SaveAsJson(string dbFilename){
    var dbFileFullName = dbPath + dbFilename + ".db";
    
    if (!File.Exists(dbFileFullName)) 
        return false;

    using (var connenction = new SQLiteConnection($"Data Source={dbFileFullName};Mode=Memory;Cache=Shared"))
    {
        var result = connenction.Query<WbwRecitationTiming>("SELECT sura, ayah, time, words FROM timings ORDER BY sura, ayah");

        var arrayList = new List<object>();

        foreach (var item in result){
            var words = item.words.Split(',');
            var wordTimes = words.Select(w=>w.Split(':').Select(s=>Convert.ToInt32(s)));
            arrayList.Add(new object[]{ item.sura, item.ayah, item.time, wordTimes, "newLine" });
        }

        var resultJson = JsonSerializer.Serialize(arrayList, jsonSerializerOptions)
                            .Replace(",\"newLine\"],", "]," + Environment.NewLine)
                            .Replace(",\"newLine\"","");
    
        File.WriteAllText(dbPath + "extractedJson\\" + dbFilename + ".json", resultJson);
    }

    return true;
}

SaveAsJson("mishari_alafasy");

#### Recitation Timing (Ayat by Ayat)

In [5]:
var dbPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\";

record RecitationTiming(Int32 sura, Int32 ayah, Int32 time);

bool SaveAsJson(string dbFilename){
    var dbFileFullName = dbPath + dbFilename + ".db";
    
    if (!File.Exists(dbFileFullName)) 
        return false;

    using (var connenction = new SQLiteConnection($"Data Source={dbFileFullName};Mode=Memory;Cache=Shared"))
    {
        var result = connenction.Query<RecitationTiming>("SELECT sura, ayah, time FROM timings ORDER BY sura, ayah");

        var arrayList = new List<object>();

        foreach (var item in result){       
            if (item.ayah != 999)     
                arrayList.Add(new object[]{ item.sura, item.ayah, item.time, "newLine" });
        }

        var resultJson = JsonSerializer.Serialize(arrayList, jsonSerializerOptions)
                            .Replace(",\"newLine\"],", "]," + Environment.NewLine)
                            .Replace(",\"newLine\"","");
    
        File.WriteAllText(dbPath + "extractedJson\\" + dbFilename + ".json", resultJson);
    }

    return true;
}

SaveAsJson("muaiqly_kfgqpc");

### Download timing files form qurancdn.com

Reciters (id, name)

- 1 abdul_baset/mujawwad
- 2 abdul_baset/murattal
- 3 abdurrahmaan_as_sudais/murattal
- 4 abu_bakr_shatri/murattal
- 5 hani_ar_rifai/murattal
- 6 khalil_al_husary/murattal
- 7 mishari_al_afasy/murattal
- 8 siddiq_al-minshawi/mujawwad
- 9 siddiq_minshawi/murattal
- 10 saud_ash-shuraym/murattal
- 11 abdul_muhsin_alqasim
- 12 khalil_al_husary/muallim
- 13 sa3d_al-ghaamidi/complete/
- 14 fares
- 17 sahl_yaaseen
- 18 salaah_bukhaatir/
- 19 ahmed_ibn_3ali_al-3ajamy/
- 43 salahbudair/
- 44 aziz_alili/
- 58 mishaari_w_ibrahim_walk_si/
- 66 abdullah_basfar_w_ibrahim_walk_si/
- 88 mostafa_ismaeel/
- 91 mohammad_altablawi/
- 95 mishaari_california/
- 97 yasser_ad-dussary/
- 104 nasser_bin_ali_alqatami/
- 122 mahmood_khaleel_al-husaree_iza3a/
- 124 abdullah_matroud/
- 126 ahmad_nauina/
- 127 akram_al_alaqmi/
- 128 ali_hajjaj_alsouasi/
- 129 mahmood_ali_albana/
- 158 ali_jaber/
- 159 maher_almu3aiqly/year1440/
- 160 bandar_baleela/complete/
- 161 khalifah_taniji/murattal
- 162 abdullaah_3awwaad_al-juhaynee/
- 163 abdullaah_basfar/
- 168 siddiq_minshawi/kids_repeat
- 169 muhammad_jibreel/complete/
- 170 khalid_jalil/murattal/mp3
- 172 hadi_toure/mp3
- 173 mishari_al_afasy/streaming/mp3
- 174 yasser_ad-dussary/mp3

In [1]:
#r "nuget: System.Net.Http.Json, *-*"

Installed Packages System.Net.Http.Json, 9.0.0-preview.5.24306.7

In [2]:
using System.IO;
using System.Text.Json;
using System.Net.Http;
using System.Net.Http.Json;
using System.Text.Encodings.Web;
using System.Text.Unicode;

var jsonSerializerOptions = new JsonSerializerOptions
{
    Encoder = JavaScriptEncoder.Create(UnicodeRanges.All),
    WriteIndented = true
};

var srcPath = @"D:\Faruque\Projects\quranpwa\src\assets\";
var quranDataJson = File.ReadAllText(srcPath + "quran-data.json");

public class QuranData
{
    public List<List<object>> suras { get; set; }
    public List<List<int>> hizb_quarters { get; set; }
    public List<List<int>> manzils { get; set; }
    public List<List<int>> rukus { get; set; }
    public List<List<int>> pages { get; set; }
    public List<List<object>> sajdas { get; set; }
    public List<List<int>> juzs { get; set; }
}

var quranData = JsonSerializer.Deserialize<QuranData>(quranDataJson);

// Root myDeserializedClass = JsonConvert.DeserializeObject<Root>(myJsonResponse);
public class AudioFile
{
    public int id { get; set; }
    public int chapter_id { get; set; }
    public object file_size { get; set; }
    public string format { get; set; }
    public string audio_url { get; set; }
    public int? duration { get; set; }
    public List<VerseTiming> verse_timings { get; set; }
}

public class Root
{
    public List<AudioFile> audio_files { get; set; }
}

public class VerseTiming
{
    public string verse_key { get; set; }
    public int timestamp_from { get; set; }
    public int timestamp_to { get; set; }
    public int? duration { get; set; }
    public List<List<double>> segments { get; set; }
}

#### Download from the server

In [7]:
var downloadPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn";

string DownloadTimingFile(int reciterId, string reciterName) 
{
    Console.WriteLine($"Downloading Timing File for " + reciterName);

    var urlBase = $"https://api.qurancdn.com/api/qdc/audio/reciters/{reciterId}/audio_files?segments=true&chapter=";
    var downloadFileName = downloadPath + $"\\{reciterName}_audioTimingFiles.json";

    if (File.Exists(downloadFileName)){
        Console.WriteLine($"File Exists!");
        return downloadFileName;
    }

    var audioTimingFiles = new List<AudioFile>();

    for(int i = 0; i < quranData.suras.Count; i++){
        var sura = quranData.suras[i];
        var serial = i + 1;
        Console.WriteLine($"Downloading sura: " + serial);

        using (var httpClient = new HttpClient()) {
            var apiUrl = urlBase + serial;
            var apiResponse = httpClient.GetAsync(apiUrl).Result;
            var apiContent = apiResponse.Content.ReadFromJsonAsync<Root>().Result;
            audioTimingFiles.AddRange(apiContent.audio_files);
        }
    }

    var resultJson = JsonSerializer.Serialize(audioTimingFiles, jsonSerializerOptions);

    File.WriteAllText(downloadFileName, resultJson);

    Console.WriteLine($"Download completed for " + reciterName);
    
    return downloadFileName;
}


#### Process from file

In [8]:
public class RecitaionTiming {
    public int sura;
    public int ayat;
    public int timeStart;
    public int duration;
    public List<List<double>> wordTimings;
}

string ProcessDownloadedFile(string downloadFileName, string reciterName)
{
    var timingJson = File.ReadAllText(downloadFileName);
    var audioTimingFiles = JsonSerializer.Deserialize<List<AudioFile>>(timingJson);

    var recitaionTimings = new List<RecitaionTiming>();

    foreach(var audioTimingFile in audioTimingFiles) {
        recitaionTimings.AddRange(audioTimingFile.verse_timings.Select(verse_timing=> {
            var verse_key = verse_timing.verse_key.Split(':');
            return new RecitaionTiming { 
                sura = int.Parse(verse_key[0]),
                ayat = int.Parse(verse_key[1]),
                timeStart = verse_timing.timestamp_from,
                duration = verse_timing.duration ?? verse_timing.timestamp_to - verse_timing.timestamp_from,
                wordTimings = verse_timing.segments
            };
        }));
    }

    var arrayList = new List<object>();

    foreach (var item in recitaionTimings){
        arrayList.Add(new object[]{ item.sura, item.ayat, item.timeStart, item.duration, item.wordTimings, "newLine" });
    }

    var resultJson = JsonSerializer.Serialize(arrayList)
                        .Replace(",\"newLine\"],", "]," + Environment.NewLine)
                        .Replace(",\"newLine\"","");

    var parsedFileName = downloadPath + $"\\{reciterName}.json";
    File.WriteAllText(parsedFileName, resultJson);

    return parsedFileName;
}


In [10]:
var reciterDict = new Dictionary<int, string>();
reciterDict.Add(1, "abdul_baset_mujawwad");
reciterDict.Add(2, "abdul_baset_murattal");
reciterDict.Add(3, "abdur-rahman_as-sudais");
reciterDict.Add(4, "abu_bakr_shatri");
// reciterDict.Add(5, "hani_ar_rifai");
// reciterDict.Add(6, "khalil_al_husary_murattal");
reciterDict.Add(7, "mishari_alafasy");
reciterDict.Add(8, "siddiq_al-minshawi_mujawwad");
reciterDict.Add(9, "siddiq_al-minshawi_murattal");
reciterDict.Add(10, "saud_ash-shuraym");
// reciterDict.Add(11, "abdul_muhsin_alqasim");
// reciterDict.Add(12, "khalil_al_husary_muallim");
reciterDict.Add(13, "saad_al-ghamidi");
// reciterDict.Add(14, "fares");
// reciterDict.Add(17, "sahl_yaaseen");
// reciterDict.Add(18, "salaah_bukhaatir");
// reciterDict.Add(19, "ahmed_ibn_3ali_al-3ajamy");
// reciterDict.Add(43, "salahbudair");
// reciterDict.Add(44, "aziz_alili");
// reciterDict.Add(58, "mishaari_w_ibrahim_walk_si");
// reciterDict.Add(66, "abdullah_basfar_w_ibrahim_walk_si");
// reciterDict.Add(88, "mostafa_ismaeel");
// reciterDict.Add(91, "mohammad_altablawi");
reciterDict.Add(95, "mishari_alafasy_california");
// reciterDict.Add(97, "yasser_ad-dussary");
// reciterDict.Add(104, "nasser_bin_ali_alqatami");
// reciterDict.Add(122, "mahmood_khaleel_al-husaree_iza3a");
// reciterDict.Add(124, "abdullah_matroud");
// reciterDict.Add(126, "ahmad_nauina");
// reciterDict.Add(127, "akram_al_alaqmi");
// reciterDict.Add(128, "ali_hajjaj_alsouasi");
// reciterDict.Add(129, "mahmood_ali_albana");
// reciterDict.Add(158, "ali_jaber");
// reciterDict.Add(159, "maher_almu3aiqly_year1440");
// reciterDict.Add(160, "bandar_baleela");
reciterDict.Add(161, "khalifah_taniji");
// reciterDict.Add(162, "abdullaah_3awwaad_al-juhaynee");
// reciterDict.Add(163, "abdullaah_basfar");
// reciterDict.Add(168, "siddiq_minshawi_kids_repeat");
// reciterDict.Add(169, "muhammad_jibreel");
// reciterDict.Add(170, "khalid_jalil_murattal");
// reciterDict.Add(172, "hadi_toure");
// reciterDict.Add(173, "mishari_al_afasy_streaming");
// reciterDict.Add(174, "yasser_ad-dussary");

foreach (var item in reciterDict) {
    var downloadFileName = DownloadTimingFile(item.Key, item.Value);

    var parsedFileName = ProcessDownloadedFile(downloadFileName, item.Value);

    Console.WriteLine(parsedFileName);
}

File Exists!
D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn\abdul_baset_mujawwad.json
File Exists!
D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn\abdul_baset_murattal.json
File Exists!
D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn\abdur-rahman_as-sudais.json
File Exists!
D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn\abu_bakr_shatri.json
File Exists!
D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn\mishari_alafasy.json
File Exists!
D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn\siddiq_al-minshawi_mujawwad.json
File Exists!
D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn\siddiq_al-minshawi_murattal.json
File Exists!
D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn\saud_ash-shuraym.json
File Exists!
D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn\saad_al-ghamidi.json
File Exists!
D:\Databa

### Downlaod from quran.gov.bd

In [3]:
using System.IO;
using System.Text.Json;
using System.Net;

var srcPath = @"D:\Faruque\Projects\quranpwa\src\assets\";
var quranDataJson = File.ReadAllText(srcPath + "quran-data.json");

public class QuranData
{
    public List<List<object>> suras { get; set; }
    public List<List<int>> hizb_quarters { get; set; }
    public List<List<int>> manzils { get; set; }
    public List<List<int>> rukus { get; set; }
    public List<List<int>> pages { get; set; }
    public List<List<object>> sajdas { get; set; }
    public List<List<int>> juzs { get; set; }
}

var quranData = JsonSerializer.Deserialize<QuranData>(quranDataJson);

record DownloadableFile(string url, string filename);

var downlaodableFiles = new List<DownloadableFile>();

for(int i = 0; i < quranData.suras.Count; i++){
    var sura = quranData.suras[i];
    var serial = i + 1;
    //var start = (int)sura[0];
    var ayas = int.Parse(sura[1].ToString());

    for(int j = 1; j <= ayas; j++){
        downlaodableFiles.Add( new($"{serial}/{serial}-{j}.mp3", $"{serial.ToString().PadLeft(3, '0')}{j.ToString().PadLeft(3, '0')}.mp3"));
    }
}

var urlBase = "http://quran.gov.bd/quran/Sound/english";
var downloadPath = @"D:\Faruque\Projects\QuranAudio\mahmudur_rahman_english_quran.gov.bd";

foreach (var downlaodableFile in downlaodableFiles){
    string url = $"{urlBase}/{downlaodableFile.url}";
    string filePath = $"{downloadPath}\\{downlaodableFile.filename}";

    if (File.Exists(filePath))
        continue;

    using (var client = new WebClient())
    {
        client.DownloadFile(url, filePath);
    }
    Console.WriteLine($"Downloaded file: {downlaodableFile}");
}

Console.WriteLine($"Download completed");


Downloaded file: DownloadableFile { url = 28/28-74.mp3, filename = 028074.mp3 }
Downloaded file: DownloadableFile { url = 28/28-75.mp3, filename = 028075.mp3 }
Downloaded file: DownloadableFile { url = 28/28-76.mp3, filename = 028076.mp3 }
Downloaded file: DownloadableFile { url = 28/28-77.mp3, filename = 028077.mp3 }
Downloaded file: DownloadableFile { url = 28/28-78.mp3, filename = 028078.mp3 }
Downloaded file: DownloadableFile { url = 28/28-79.mp3, filename = 028079.mp3 }
Downloaded file: DownloadableFile { url = 28/28-80.mp3, filename = 028080.mp3 }
Downloaded file: DownloadableFile { url = 28/28-81.mp3, filename = 028081.mp3 }
Downloaded file: DownloadableFile { url = 28/28-82.mp3, filename = 028082.mp3 }
Downloaded file: DownloadableFile { url = 28/28-83.mp3, filename = 028083.mp3 }
Downloaded file: DownloadableFile { url = 28/28-84.mp3, filename = 028084.mp3 }
Downloaded file: DownloadableFile { url = 28/28-85.mp3, filename = 028085.mp3 }
Downloaded file: DownloadableFile { url 